In [ ]:
import numpy as np
import pathlib
import random
import shutil
import torch
import os
import matplotlib.pyplot as plt
import torch.nn as nn
%matplotlib inline

from os import path
from torchvision import transforms, datasets, models
from torch import optim

In [ ]:
os.listdir("../input/archive")

In [ ]:
# define the transform
train_transform = transforms.Compose([
                                transforms.RandomRotation(40),
                                transforms.RandomResizedCrop(224), # most pretrained model have this size of image
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], # Normalize by mean and std
                                                            [0.229, 0.224, 0.225])])

val_transform = transforms.Compose([
                        transforms.RandomResizedCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], # Normalize by mean and std
                                                            [0.229, 0.224, 0.225])])
# load the data
data_path = "../input/archive/"
train_data = datasets.ImageFolder(data_path + "train1", transform = train_transform)
val_data = datasets.ImageFolder(data_path + "val", transform=val_transform)


In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=16)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

In [ ]:
def imshow(image, ax=None, normalize=True):
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        # if the data loader has transform.normalize
        # undo preprocessing
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    
    return ax

In [ ]:
imshow(images[1]);
imshow(images[4]);
imshow(images[6]);

In [ ]:
model = models.densenet201(pretrained=True)

In [ ]:
for param in model.parameters():
    param.required_grad = False

In [ ]:
# change the classifier
from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
              ('fc1', nn.Linear(1920, 500)),
              ('relu1', nn.ReLU()),
              ('dropout1', nn.Dropout(p=0.2)),
              ('fc2', nn.Linear(500, 256)),
              ('relu2', nn.ReLU()),
              ('dropout2', nn.Dropout(p=0.2)),
              ('fc3', nn.Linear(256, 10)),
              ('output', nn.LogSoftmax(dim=1))
              
]))
model.classifier = classifier

In [ ]:
 #Train either on GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
criterion = nn.NLLLoss()

optimizer = optim.SGD(model.classifier.parameters(), lr = 0.01, momentum=0.9)

model.to(device)

In [ ]:
epochs = 15

for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        validation_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            model.eval()
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                validation_loss += loss.item()
                
                ps = torch.exp(outputs)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()

        print("Epoch: {}/{}.. ".format(epoch+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Valid Loss: {:.3f}.. ".format(validation_loss/len(val_loader)),
              "Valid Accuracy: {:.3f}".format(accuracy/len(val_loader)))


In [ ]:
model.class_to_idx = train_data.class_to_idx

In [ ]:
torch.save({'epoch': epoch + 1,
            "state_dict":model.state_dict(),
           "class_to_idx":model.class_to_idx}, 'classifier.pth')